In [ ]:
import hsfs
connection = hsfs.connection(
    host="8ee70ae0-9c91-11ec-a00c-8db997ac0d25.cloud.hopsworks.ai", # change 'host' to point to your cluster
    project="fsbook",   # create a project called 'fsbook' in your Hopsworks cluster
    engine="hive",
    secrets_store="local",
    api_key_file="./api-key.txt"
)
fs = connection.get_feature_store()

In [1]:
import pandas as pd

df = pd.read_csv("../../data/ecommerce/order_item.csv")
df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [ ]:
order_item_fg = fs.create_feature_group(
                        name="order_item_fg",
                        primary_key=["order_id", "order_item_id"],
                        version=1,
                        description="Order items",
                        online_enabled=True
)

order_item_fg.save(df)

In [ ]:

query = product_fg.select_all().join(order_item_fg.select_all())

feature_view = fs.create_feature_view(
    name="product_orders",
    description="Training data for a products and orders model",
    label="price",
    query=query
)

df = feature_view.get_training_data(version=1)
df

In [ ]:
# Here, we use the built-in storage connector to Hopsworks that will store the training data in Hopsworks.
# Hopsworks includes "hopsfs", a scale out file system with a HDFS API and a object storage backend.
# You can create your own storage connector to save your training data in a different storage backend.

hopsfs = fs.get_storage_connector("fsbook_Training_Datasets")

# This will take a few mins to run, as it will spin up a Spark job in Hopsworks to join the features
# together and write out the training data as files in .tfrecord file format (TensorFlow Records)
# on the file system in Hopsworks.
feature_view.create_training_data(
    version=1,
    connector=hopsfs,
    data_format="tfrecord"
)